<a href="https://colab.research.google.com/github/rudaruda/Chatbots/blob/main/ChatBot_Python_NLTK_ROBELE_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bibliotecas necessárias

In [93]:
import io, random, string, warnings
import numpy as np
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

### Instalando Pacotes do NLTK

In [94]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download() # for downloading packages
nltk.download('popular', quiet=True) # for downloading packages
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Intalando Pcotes de Stem (em Portugues)

In [95]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("portuguese")
#englishStemmer.stem("continha")

## Carregando o conhecimento do CHATBOT

In [96]:
f=open('chatbot2.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase

## Tockenização

In [97]:
sent_tokens = nltk.sent_tokenize(raw)# converte List->Sentença 
word_tokens = nltk.word_tokenize(raw)# converte List->Palavra
stopwords_ok = set(stopwords.words('portuguese') + list(punctuation)) # coleção de stopwords
print('Exemplo "sent_tokens":\n',sent_tokens[:2])
print('Exemplo "word_tokens":\n',word_tokens[:5])
print('Exemplo "stopwords_ok"\n',stopwords_ok)

Exemplo "sent_tokens":
 ['chatbot (ou chatterbot) é um programa de computador que tenta simular um ser humano na conversação com as pessoas.', '[1] o objetivo é responder as perguntas de tal forma que as pessoas tenham a impressão de estar conversando com outra pessoa e não com um programa de computador.']
Exemplo "word_tokens":
 ['chatbot', '(', 'ou', 'chatterbot', ')']
Exemplo "stopwords_ok"
 {'na', 'formos', 'terei', 'já', 'meu', 'teus', 'estavam', 'houveriam', 'fosse', 'estão', 'estivermos', 'das', 'essas', 'sejamos', 'deles', 'tenham', 'houverem', 'dela', '+', 'lhes', 'ela', 'suas', 'hei', 'seu', 'teve', 'fui', 'tivermos', 'isso', 'tivessem', '!', 'aquelas', '[', 'somos', 'sejam', 'nossos', 'aquilo', 'como', 'sua', 'um', 'lhe', 'por', 'é', 'os', 'às', '$', 'da', 'te', 'mas', 'este', 'meus', ')', 'hajam', 'à', '-', '&', 'nem', 'estejamos', 'tivéramos', 'teria', 'vocês', 'também', 'sou', 'estamos', 'estive', 'fomos', '~', 'se', '<', 'no', 'estejam', 'estiverem', 'for', 'nos', 'hajam

## Pre-processamento
WordNet é um dicionario semantico em Inglês(pode ser um ponto fraco)<br>
Por isso, vamos tentar com altertiva de Stemming em portugues porque existem variações de linguistica

In [101]:
lemmer = nltk.stem.WordNetLemmatizer()
lemmer2 = englishStemmer
def LemTokens(tokens):
    #return [lemmer.lemmatize(token) for token in tokens]
    return [lemmer2.stem(token) for token in tokens] #ALTERNATIVA AQUI
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## Encontro de palavras (Match Keywords)

In [104]:
GREETING_INPUTS = ("olá","como vai você","como vai","e ai","blz","bele","beleza","bom dia","boa tarde","boa noite","dia","tarde","noite","hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["aew", "o que manda hj?", "...hummm...", "to aqui!", "oi", "To di boa! Agora q vc chegou :-)"]

# Checking for greetings
def greeting(sentence):
    """Pergunta entrada respondida imediatamente"""
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## Gerando a resposta (Function Generating Response)

### Grupos de palavras (Bag of Words)
Depois do processamento da frase, nos precisamos transforma Texto em Vetor de Números.
Os grupos de palavras são a representação de textos que estão descritos as ocorrencias que estão contidas no docjmento. Cumprindo 2 requisitos:
- Um vocabulario de palavras conhecidas.
- Avaliação de palavras conhecidas.

A "inteligencia" por trás são esses "Grupos de Palavras"/"Bag of Words". Com esse metodo é possivel obter algum aprendizado sobre conteúdo do documento.

Abrodagem TF-IDF
É preciso "tomar cuidado" com palavras muito frequentes que podem ter menos conteúdo.
Outro problema deste método é ele da mais importancia aos paragrafos mais longos.

Uma solução para esse problema é re-escalr a a frequencia das palavras que aparecem em todos os paragrafos com uma penalização. O nome desta abordagem é "Term Frequency-Inverse Document Frequency", ou "TF-IDF for short".

In [ ]:
# Generating response
def response(user_response):
    robo_response=''
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stopwords_ok)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Como eh? humm?"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

# Finalmente
Vamos poder ver o rôbo em ação:

In [100]:
flag=True
print("ROBELE: Meu nome é ROBELE. Eu respondo coisas sobre Chatbots. Quando quiser vaza, digita Xau!")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(not user_response in ['bye','xau']):
        if(user_response in ['vlw','obrigado'] or user_response=='flww' ):
            flag=False
            print("ROBELE: e aew...")
        else:
            if(greeting(user_response)!=None):
                print("ROBELE: "+greeting(user_response))
            else:
                sent_tokens.append(user_response)
                word_tokens=word_tokens+nltk.word_tokenize(user_response)
                final_words=list(set(word_tokens))
                print("ROBELE: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBELE: Xau! Pa tu tb")    

ROBELE: Meu nome é ROBELE. Eu respondo coisas sobre Chatbots. Quando quiser vaza, digita Xau!
quando foi criado os chatbots
ROBELE: os chatbots alice e bob criados pelo facebook estavam criando um idioma próprio e passaram a agir de forma diferente da que foi descrita por seus programadores no início do projeto.
o que é chatbot
ROBELE: chatbot (ou chatterbot) é um programa de computador que tenta simular um ser humano na conversação com as pessoas.
como crio um chatbot
ROBELE: os chatbots alice e bob criados pelo facebook estavam criando um idioma próprio e passaram a agir de forma diferente da que foi descrita por seus programadores no início do projeto.
como faço chatbot
ROBELE: chatbot (ou chatterbot) é um programa de computador que tenta simular um ser humano na conversação com as pessoas.
como é construido um chat
ROBELE: [6]

para criar chatbots é necessário uma plataforma de sistema computacional, geralmente fornecidos no modelo de computação na nuvem (cloud computing) e surgira

# Conclusão
- Tecnicamente é uma abordagem muito simples de ser aplicada sobre qualquer tipo de documento de texto que contenha estrutura de paragrafos.
- Porém há limitações, dependendo da format como aplicaou a "Lematização" pode ser que a sua pergunta não seja compreendida pela aplicação.
- Como vimos no teste da aplicação, foi necessário que eu refizesse a mesma pergunta de três formas diferentes "qual a desvatagem", "qual a desvatagem", "e a desvantagem". Esses três termos queriam dizer a mesma coisa, porém com erro de português ou conter termos "desconhecidos" no "Bag of words" podem deixar a aplicação com grande limitações.
- Vejo, já muitas aberturas, para melhorar o algoritmo de "Lematização" e evoluir a aplicação a um nível comercialmente aceitavél. Como experiência alcancei o objetivo de construir um ChatBot extremamente simples e poucas horas.


# Bibliografia

- https://github.com/Spandan-Madan/Me_Bot
- https://stackoverflow.com/questions/65117858/python-chatbot-using-excel
- https://github.com/parulnith/Building-a-Simple-Chatbot-in-Python-using-NLTK
- https://lars76.github.io/2018/05/08/portuguese-lemmatizers.html
- https://stackoverflow.com/questions/57340142/user-warning-your-stop-words-may-be-inconsistent-with-your-preprocessing
- https://medium.com/@viniljf/utilizando-processamento-de-linguagem-natural-para-criar-um-sumariza%C3%A7%C3%A3o-autom%C3%A1tica-de-textos-775cb428c84e
- https://www.geeksforgeeks.org/python-stemming-words-with-nltk/